# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [11]:
from __future__ import division
import turicreate
import math
import string

# Data preperation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [3]:
products = turicreate.SFrame('amazon_baby.csv')

Finished parsing file /home/ppratikbp/Coursera Machine Learning/C3_1_Linear Classifiers & Logistic Regression/amazon_baby.csv

Parsing completed. Parsed 100 lines in 1.12613 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 105953 lines. Lines per second: 90536.5

Finished parsing file /home/ppratikbp/Coursera Machine Learning/C3_1_Linear Classifiers & Logistic Regression/amazon_baby.csv

Parsing completed. Parsed 183531 lines in 1.40546 secs.

Now, let us see a preview of what the dataset looks like.

In [4]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [5]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'review': 'A favorite in our house!',
 'rating': 5}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [24]:
def remove_punctuation(text):
    import string
    return text.translate(string.punctuation) 

review_without_puctuation = products['review'].apply(remove_punctuation)
products['word_count'] = turicreate.text_analytics.count_words(review_without_puctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [25]:
products[269]['word_count']

{'baby': 1.0,
 'has': 1.0,
 'anyone': 1.0,
 'book': 1.0,
 'sears': 1.0,
 'saver': 1.0,
 'use': 1.0,
 'no': 1.0,
 'older': 1.0,
 'now': 1.0,
 'dr': 1.0,
 'put': 1.0,
 'slipped': 1.0,
 'life': 1.0,
 'just': 1.0,
 'her': 2.0,
 'very': 1.0,
 'bring': 1.0,
 'great': 2.0,
 'up': 1.0,
 'way': 1.0,
 'sit': 2.0,
 'who': 1.0,
 'at': 1.0,
 'longer': 1.0,
 'down': 3.0,
 'lie': 2.0,
 'wanted': 1.0,
 'then': 1.0,
 'far': 1.0,
 'off': 1.0,
 'legs': 1.0,
 'daughter': 1.0,
 'or': 1.0,
 'husbad': 1.0,
 'too': 1.0,
 'and': 4.0,
 'only': 2.0,
 'do': 1.0,
 'fit': 1.0,
 'not': 4.0,
 'size': 1.0,
 'of': 2.0,
 '6': 1.0,
 'with': 1.0,
 'she': 4.0,
 'it': 8.0,
 'this': 2.0,
 'small': 1.0,
 'after': 1.0,
 'my': 1.0,
 'myself': 1.0,
 'the': 4.0,
 'adjustment': 1.0,
 'disagree': 1.0,
 'that': 2.0,
 'is': 2.0,
 'say': 1.0,
 '5': 1.0,
 '4': 1.0,
 'i': 5.0,
 'bought': 1.0,
 'one': 1.0,
 'rid': 1.0,
 'fussy': 2.0,
 'frame': 1.0,
 'sleep': 1.0,
 'our': 1.0,
 'all': 3.0,
 'to': 6.0,
 'was': 7.0,
 'a': 2.0,
 'could': 2.0

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [26]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [27]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5,"{'recommend': 1.0,'highly': 1.0, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5,"{'quilt': 1.0, 'of': 1.0,'the': 1.0, 'than': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5,"{'tool': 1.0, 'clever':1.0, 'approach': 2.0, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5,"{'rock': 1.0, 'many':1.0, 'headaches': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5,"{'thumb': 1.0, 'or': 1.0,'break': 1.0, 'trying': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4,"{'for': 1.0, 'barnes':1.0, 'at': 1.0, 'is': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5,"{'right': 1.0, 'because':1.0, 'questions': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5,"{'like': 1.0, 'and': 1.0,'changes': 1.0, 'the': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4,"{'tracker': 1.0, 'now':1.0, 'its': 1.0, 'sti ...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [28]:
train_data, test_data = products.random_split(.8, seed=1)
print(len(train_data))
print(len(test_data))

133416
33336


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [30]:
sentiment_model = turicreate.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.656037  | 3.535792     | 0.938621          |

| 3         | 12       | 2.656037  | 3.844610     | 0.927535          |

| 4         | 14       | 1.341935  | 4.316727     | 0.945441          |

| 9         | 20       | 1.341935  | 5.936904     | 0.977387          |

+-----------+----------+-----------+--------------+-------------------+

In [31]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 57446
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 57445

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 10
Solver status                  : Completed (Iteration limit reached).
Training time (sec)            : 3.768

Settings
--------
Log-likelihood                 : 9620.6091

Highest Positive Coefficients
-----------------------------
word_count[arghhhhhh]          : 48.5657
word_count[screencons]         : 32.0635
word_count[pinkjeep]           : 28.0567
word_count[joovys]             : 27.3089
word_count[ozone]              : 27.1083

Lowest Negative Coefficients
----------------------------
word_count[impulsejeep]        

**Aside**. You may get an warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [32]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

In [33]:
weights

name,index,class,value,stderr
(intercept),None,1,1.2357054152241465,None
word_count,recommend,1,0.3548384562812628,None
word_count,highly,1,1.8294355809197083,None
word_count,disappointed,1,-3.3961291177666655,None
word_count,love,1,1.536354186976046,None
word_count,it,1,0.0218384126296374,None
word_count,planet,1,-0.789011182123935,None
word_count,and,1,0.05535496177167002,None
word_count,bags,1,0.26099757137246477,None
word_count,wipes,1,-0.02055397480148092,None


There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [34]:
weights.num_rows()

57446

In [36]:
num_positive_weights = (weights['value'] >= 0).sum()
num_negative_weights = (weights['value'] < 0).sum()

print("Number of positive weights: %s " % num_positive_weights)
print("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 40270 
Number of negative weights: 17176 


**Quiz question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [37]:
sample_test_data = test_data[10:13]
print(sample_test_data['rating'])
sample_test_data

[5, 2, 1]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5,"{'again': 1.0, 'book':1.0, 'same': 1.0, ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2,"{'peeling': 1.0, '5':1.0, 'about': 1.0, 'f ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1,"{'on': 1.0, 'is': 1.0,'back': 1.0, ...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [38]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [39]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [41]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print(scores)

[6.593328532772327, -5.387775160092742, -14.78561326787155]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [42]:
def class_predictions(scores):
    """ make class predictions
    """
    preds = []
    for score in scores:
        if score > 0:
            pred = 1
        else:
            pred = -1
        preds.append(pred)
    return preds

In [43]:
class_predictions(scores)

[1, -1, -1]

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [47]:
print("Class predictions according to GraphLab Create:")
print(sentiment_model.predict(sample_test_data))

Class predictions according to GraphLab Create:
[1, -1, -1]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [48]:
def calculate_proba(scores):
    """ Calculate the probability predictions from the scores.
    """
    proba_preds = []
    for score in scores:
        proba_pred =  1 / (1 + math.exp(-score))
        proba_preds.append(proba_pred)
    return proba_preds

calculate_proba(scores)

[0.9986323988645204, 0.004551325025891775, 3.7904396084479313e-07]

**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [49]:
print("Class predictions according to GraphLab Create:" )
print(sentiment_model.predict(sample_test_data, output_type='probability'))

Class predictions according to GraphLab Create:
[0.9986323988645204, 0.004551325025891774, 3.790439608447928e-07]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [50]:
# probability predictions on test_data using the sentiment_model
test_data['proba_pred'] = sentiment_model.predict(test_data, output_type='probability')
test_data

name,review,rating,word_count,sentiment
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4,"{'in': 1.0, 'pages': 1.0,'out': 1.0, 'run': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4,"{'tracker': 1.0, 'now':1.0, 'its': 1.0, 'sti ...",1
Nature's Lullabies FirstYear Sticker Calendar ...,"I love this littlecalender, you can keep ...",5,"{'too': 1.0, 'stickers':1.0, 'illustrations': ...",1
Nature's Lullabies SecondYear Sticker Calendar ...,"I had a hard time findinga second year calendar, ...",5,"{'reference': 1.0,'have': 1.0, 'out': 1.0, ...",1
"Lamaze Peekaboo, I LoveYou ...","One of baby's first andfavorite books, and i ...",4,"{'typical': 1.0, 'your':1.0, 'the': 1.0, ...",1
"Lamaze Peekaboo, I LoveYou ...",My son loved this book asan infant. It was ...,5,"{'farm': 1.0, 'out': 1.0,'say': 1.0, 'again': ...",1
"Lamaze Peekaboo, I LoveYou ...",Our baby loves this book& has loved it for a ...,5,"{'own': 1.0, 'his': 1.0,'on': 1.0, 'a': 1.0, ...",1
"SoftPlay Giggle JiggleFunbook, Happy Bear ...",This bear is absolutelyadorable and I would ...,2,"{'granddaughter': 1.0,'kenzie': 1.0, 'my': ...",-1
SoftPlay Peek-A-BooWhere's Elmo A Childr ...,I bought two for recentbaby showers! The book ...,5,"{'elmo': 1.0, 'love':1.0, 'for': 2.0, 'is': ...",1
Baby's First Year UndatedWall Calendar with ...,I searched high and lowfor a first year cale ...,5,"{'that': 1.0, 'happy':1.0, 'memories': 1.0, ...",1


In [51]:
test_data['name','proba_pred'].topk('proba_pred', k=20).print_rows(20)

+-------------------------------+------------+
|              name             | proba_pred |
+-------------------------------+------------+
|  Fisher-Price Deluxe Jumperoo |    1.0     |
| Itzbeen Pocket Nanny Baby ... |    1.0     |
| Ameda Purely Yours Breast ... |    1.0     |
| Shermag Glider Rocker Comb... |    1.0     |
| Kolcraft Contours Options ... |    1.0     |
| Vulli Sophie the Giraffe T... |    1.0     |
| Cloud b Sound Machine Soot... |    1.0     |
| JP Lizzy Chocolate Ice Cla... |    1.0     |
| Fisher-Price Rainforest Me... |    1.0     |
| Lilly Gold Sit 'n' Stroll ... |    1.0     |
| Basic Comfort Handy's Plus... |    1.0     |
| Cloud b Twilight Constella... |    1.0     |
| Britax Decathlon Convertib... |    1.0     |
| Parent Units 2 Pack Equipm... |    1.0     |
| Stork Craft Beatrice Combo... |    1.0     |
|   Capri Stroller - Red Tech   |    1.0     |
| Peg Perego Primo Viaggio C... |    1.0     |
| Summer Infant Complete Nur... |    1.0     |
|  BABYBJORN 

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this excercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [52]:
test_data['name','proba_pred'].topk('proba_pred', k=20, reverse=True).print_rows(20)

+-------------------------------+------------------------+
|              name             |       proba_pred       |
+-------------------------------+------------------------+
| Jolly Jumper Arctic Sneak ... | 1.511634431202698e-61  |
| The First Years True Choic... | 9.039884978235666e-38  |
| VTech Communications Safe ... | 1.6865759536577354e-28 |
| Safety 1st High-Def Digita... | 1.3183706636162261e-23 |
| Snuza Portable Baby Moveme... | 1.5433840772681288e-22 |
| Samsung SEW-3037W Wireless... | 2.6871019419516235e-22 |
| Levana Safe N'See Digital ... | 3.975935932641656e-22  |
|  KidCo Center Gateway - White | 5.4469056173137915e-21 |
|    Fisher-Price Royal Potty   | 4.813331375756995e-20  |
| Cloth Diaper Sprayer--styl... | 1.5896444798740093e-19 |
| Fisher-Price Ocean Wonders... | 3.4606295566332727e-19 |
| Adiri BPA Free Natural Nur... | 6.870349963606475e-19  |
| Stork Craft Hoop Glider an... | 1.3270829292916024e-18 |
| Peg-Perego Tatamia High Ch... | 1.4384654960192904e-18

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifer. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [55]:
# Test SArray comparison
print(turicreate.SArray([1,1,1]) == sample_test_data['sentiment'])
print(sentiment_model.predict(sample_test_data) == sample_test_data['sentiment'])

[1, 0, 0]
[1, 1, 1]


In [56]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    predictions = model.predict(data)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    # compare 2 SArray, true = 1, false = 0
    num_correct = sum(predictions == true_labels)

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = num_correct/len(data)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [57]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9178365730741541

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [58]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [59]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [60]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [61]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [63]:
print(train_data[0]['word_count'])

{'recommend': 1.0, 'highly': 1.0, 'disappointed': 1.0, 'love': 1.0, 'it': 3.0, 'planet': 1.0, 'and': 3.0, 'bags': 1.0, 'wipes': 1.0, 'not': 2.0, 'early': 1.0, 'came': 1.0, 'i': 1.0, 'does': 1.0, 'wise': 1.0, 'my': 2.0, 'was': 1.0, 'now': 1.0, 'wipe': 1.0, 'holder': 1.0, 'leak': 1.0, 'keps': 1.0, 'osocozy': 1.0, 'moist': 1.0}


Since we are only working with a subet of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [65]:
print(train_data[0]['word_count_subset'])

{'disappointed': 1.0, 'love': 1.0}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [68]:
simple_model = turicreate.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.805908     | 0.867242          |

| 6         | 7        | 0.955432     | 0.867242          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.9798

Settings
--------
Log-likelihood                 : 44164.7122

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6816
word_count_subset[perfect]     : 1.5109
word_count_subset[love]        : 1.3581
(intercept)                    : 1.2933
word_count_subset[easy]        : 1.1846

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3571
wo

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [69]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8702303815694744

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [70]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.2932808921547936,0.012145978391362697
word_count_subset,disappointed,1,-2.3571037394161527,0.050063377141656346
word_count_subset,love,1,1.3580674001759985,0.03017458453252441
word_count_subset,would,1,-0.3633995388401757,0.01273370596876201
word_count_subset,well,1,0.5306735620456097,0.021255740173511815
word_count_subset,product,1,-0.3266819017512045,0.015215122005230128
word_count_subset,loves,1,1.6816120247452697,0.04841536802274801
word_count_subset,little,1,0.5150234016435977,0.021476474442059086
word_count_subset,great,1,0.9441757614322625,0.02074475204764408
word_count_subset,work,1,-0.6239930388722468,0.022918890620554185


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [71]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+---------------------+
|        name       |    index     | class |        value        |
+-------------------+--------------+-------+---------------------+
| word_count_subset |    loves     |   1   |  1.6816120247452697 |
| word_count_subset |   perfect    |   1   |   1.51092914851628  |
| word_count_subset |     love     |   1   |  1.3580674001759985 |
|    (intercept)    |     None     |   1   |  1.2932808921547936 |
| word_count_subset |     easy     |   1   |  1.1845752461137113 |
| word_count_subset |    great     |   1   |  0.9441757614322625 |
| word_count_subset |     well     |   1   |  0.5306735620456097 |
| word_count_subset |    little    |   1   |  0.5150234016435977 |
| word_count_subset |     able     |   1   |  0.1935309746584927 |
| word_count_subset |     old      |   1   | 0.08269162076978927 |
| word_count_subset |     car      |   1   | 0.05648814949816783 |
| word_count_subset |     less     |   1   | -0.20197046774142

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [73]:
simple_weights = simple_model.coefficients
positive_significant_words = simple_weights[(simple_weights['value'] > 0) & (simple_weights['name'] == "word_count_subset")]['index']
print(len(positive_significant_words))
print(positive_significant_words)

10
['love', 'well', 'loves', 'little', 'great', 'easy', 'able', 'perfect', 'old', 'car']


**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [74]:
weights.filter_by(positive_significant_words, 'index')

name,index,class,value,stderr
word_count,love,1,1.536354186976046,None
word_count,well,1,0.687347607771418,None
word_count,loves,1,1.7391243270977614,None
word_count,little,1,0.7880854558525462,None
word_count,great,1,1.380494031644942,None
word_count,easy,1,1.3083313420163138,None
word_count,able,1,0.24203333157331358,None
word_count,perfect,1,1.925802328167505,None
word_count,old,1,0.03180753872230501,None
word_count,car,1,0.19204365412564378,None


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [75]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

0.9773865203573784

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [76]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

0.8672423097679439

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this excercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [77]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9178365730741541

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [78]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8702303815694744

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [80]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

**Notes: what is majority class classifier?**
- https://www.coursera.org/learn/ml-classification/module/0Wclc/discussions/KoQDsuMOEeWFPAqzawk_tQ

It is a classifier that always picks the most frequently occurring class (the "majority class"). For example, consider spam classification with two classes: positive/"it's spam" and negative/"it's not spam". Around 90% of all email is spam, so in the case of spam classification, the positive class is the majority class. The majority class spam classifier always guesses that an email is spam. Despite the simplicity, this classifier will have around 90% accuracy!

In the first assignment, we are asked to determine the accuracy of a majority class classifier where the two classes are +1/positive review sentiment or -1/negative review sentiment. A majority class classifier is "trained" to guess the majority class of the training set. For example, if the majority of reviews in the training set are positive, then the majority class will always guess that a review is positive.


In [83]:
print((test_data['sentiment'] == +1).sum())
print((test_data['sentiment'] == -1).sum())

28095
5241


In [84]:
print((test_data['sentiment'] == +1).sum()/len(test_data['sentiment']))

0.8427825773938085
